In [2]:
import json 
import pickle
import random
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 64
max_len = 20

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 64)            64000     
                                                                 
 global_average_pooling1d (  (None, 64)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 54)                918       
                                                                 
Total params: 66230 (258.71 KB)
Trainable params: 66230 (258.71 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [7]:
history = model.fit(padded_sequences, np.array(training_labels), epochs= 500)

Epoch 1/500
4/4 [==============================] - 1s 7ms/step - loss: 3.9889 - accuracy: 0.0101
Epoch 2/500
4/4 [==============================] - 0s 8ms/step - loss: 3.9841 - accuracy: 0.0303
Epoch 3/500
4/4 [==============================] - 0s 7ms/step - loss: 3.9805 - accuracy: 0.0303
Epoch 4/500
4/4 [==============================] - 0s 8ms/step - loss: 3.9768 - accuracy: 0.0303
Epoch 5/500
4/4 [==============================] - 0s 6ms/step - loss: 3.9729 - accuracy: 0.0303
Epoch 6/500
4/4 [==============================] - 0s 6ms/step - loss: 3.9687 - accuracy: 0.0303
Epoch 7/500
4/4 [==============================] - 0s 7ms/step - loss: 3.9640 - accuracy: 0.0303
Epoch 8/500
4/4 [==============================] - 0s 6ms/step - loss: 3.9599 - accuracy: 0.0303
Epoch 9/500
4/4 [==============================] - 0s 8ms/step - loss: 3.9547 - accuracy: 0.0303
Epoch 10/500
4/4 [==============================] - 0s 7ms/step - loss: 3.9492 - accuracy: 0.0303
Epoch 11/500
4/4 [===========

In [8]:
# save the trained model
model.save("chatbot_model")

# save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chatbot_model\assets


INFO:tensorflow:Assets written to: chatbot_model\assets


In [9]:
with open("intents.json") as file:
    data = json.load(file)

def ChatBot():

    model = keras.models.load_model('chatbot_model')

    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
    
    while True:
      print("User: ", end="")
      n = input()
      if n.lower() == "quit":
          break

      result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([n]),
                                            truncating= 'post', maxlen= 20))
      tag = lbl_encoder.inverse_transform([np.argmax(result)])

      for i in data['intents']:
          if i['tag'] == tag:
              print("Bot: " + np.random.choice(i['responses']))

In [10]:
ChatBot()

User: 